<a href="https://colab.research.google.com/github/vincentjunitio00/Heart_Disease_Clustering/blob/main/Heart_Disease_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
!git clone https://github.com/vincentjunitio00/Heart_Disease_Clustering

Cloning into 'Heart_Disease_Clustering'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [5]:
def preprocessing(file):
  file = open(file, "r")

  counter = 0
  records = ''
  dataset = []
  for record in file.readlines():
    records += record.rstrip('\n') + ' '
    counter += 1
    if counter >= 10:
      dataset.append(records.rstrip())
      records = ''
      counter = 0
  return dataset

In [6]:
dataset_cleveland = preprocessing('/content/Heart_Disease_Clustering/cleveland.data')
dataset_hungarian = preprocessing('/content/Heart_Disease_Clustering/hungarian.data')
dataset_longbeach = preprocessing('/content/Heart_Disease_Clustering/long-beach-va.data')
dataset_switzer = preprocessing('/content/Heart_Disease_Clustering/switzerland.data')

In [7]:
used_columns = [0, 2, 3, 40, 17, 15, 4, 50, 11, 9, 37, 31]
used_columns_name = {0: 'id', 2: 'age', 3: 'sex', 40: 'slope', 17: 'famhist', 15:'fbs', 4:'painloc', 50: 'thal', 11: 'chol', 9: 'trestbps', 37: 'exang', 31: 'thalach'}

# Repeat here

In [8]:
df_cleveland = pd.DataFrame([data.split() for data in dataset_cleveland])
df_used_cleveland = df_cleveland[used_columns].rename(columns=used_columns_name)

f = open('/content/Heart_Disease_Clustering/processed.cleveland.data', 'r')
preprocessed_columns_name = {0:'age', 1: 'sex', 2: 'cp', 3: 'trestbps', 4: 'chol', 5: 'fbs', 6: 'restecg', 7: 'thalach', 8: 'exang', 9: 'oldpeak', 10: 'slope', 11: 'ca', 12: 'thal', 13: 'num'}

df_preprocessed_cleveland = pd.DataFrame([data.rstrip('\n').split(',') for data in f.readlines()]).rename(columns=preprocessed_columns_name)

df_used_cleveland['num'] = df_preprocessed_cleveland['num']

In [9]:
df_hungarian = pd.DataFrame([data.split() for data in dataset_hungarian])
df_used_hungarian = df_hungarian[used_columns].rename(columns=used_columns_name)

f = open('/content/Heart_Disease_Clustering/reprocessed.hungarian.data', 'r')
df_preprocessed_hungarian = pd.DataFrame([data.rstrip('\n').split(' ') for data in f.readlines()]).rename(columns=preprocessed_columns_name)

df_used_hungarian['num'] = df_preprocessed_hungarian['num']

In [10]:
df_longbeach = pd.DataFrame([data.split() for data in dataset_longbeach])
df_used_longbeach = df_longbeach[used_columns].rename(columns=used_columns_name)

f = open('/content/Heart_Disease_Clustering/processed.va.data', 'r')
preprocessed_columns_name = {0:'age', 1: 'sex', 2: 'cp', 3: 'trestbps', 4: 'chol', 5: 'fbs', 6: 'restecg', 7: 'thalach', 8: 'exang', 9: 'oldpeak', 10: 'slope', 11: 'ca', 12: 'thal', 13: 'num'}

df_preprocessed_longbeach = pd.DataFrame([data.rstrip('\n').split(',') for data in f.readlines()]).rename(columns=preprocessed_columns_name)
df_used_longbeach['num'] = df_preprocessed_longbeach['num']

In [11]:
df_switzer = pd.DataFrame([data.split() for data in dataset_switzer])
df_used_switzer = df_switzer[used_columns].rename(columns=used_columns_name)

f = open('/content/Heart_Disease_Clustering/processed.switzerland.data', 'r')
df_preprocessed_switzerland = pd.DataFrame([data.rstrip('\n').split(',') for data in f.readlines()]).rename(columns=preprocessed_columns_name)

df_used_switzerland = df_used_switzer.merge(df_preprocessed_switzerland, how='inner', on=['age', 'trestbps', 'thalach'], suffixes=('','_y'))
df_used_switzerland = df_used_switzerland[['id','age','sex','slope','famhist','fbs','painloc','thal','chol','trestbps','exang','thalach','num']]

In [12]:
df = df_used_cleveland.append([df_used_hungarian, df_used_longbeach, df_used_switzerland]).reset_index(drop=True)

In [13]:
df

,id,age,sex,slope,famhist,fbs,painloc,thal,chol,trestbps,exang,thalach,num
0,1,63,1,3,1,1,-9,6,233,145,0,150,0
1,2,67,1,2,1,0,-9,3,286,160,1,108,2
2,3,67,1,2,1,0,-9,7,229,120,1,129,1
3,4,37,1,3,1,0,-9,3,250,130,0,187,0
4,6,41,0,1,1,0,-9,3,204,130,0,172,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,4070,54,1,2,-9,-9,1,7,0,180,0,150,1
893,4071,56,1,2,-9,1,1,7,0,125,1,103,3
894,4072,56,1,2,-9,-9,0,7,0,125,0,98,2
895,4073,54,1,2,-9,-9,1,7,0,130,1,110,3
